In [11]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# 1. Carregamento e Limpeza
print("Carregando dados...")
# Tenta carregar com utf-8, se falhar usa latin1 (comum em arquivos pt-br)
try:
    df = pd.read_csv('buscape.csv')
except UnicodeDecodeError:
    df = pd.read_csv('buscape.csv', encoding='latin1')

# Garantir que é string e remover vazios
df['review_text'] = df['review_text'].astype(str)
df = df.dropna(subset=['review_text', 'polarity'])

# Mapeamento: O dataset tem 0.0 (Negativo) e 1.0 (Positivo)
# Se houver ratings neutros (ex: 3 estrelas), poderíamos tratar aqui,
# mas vamos focar no binário conforme o MVP.
def get_label(polarity):
    if polarity == 1.0:
        return "Positivo"
    else:
        return "Negativo"

df['sentiment'] = df['polarity'].apply(get_label)

print(f"Distribuição das classes:\n{df['sentiment'].value_counts()}")

# 2. Divisão Treino/Teste
X = df['review_text']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3. Pipeline de Machine Learning
# TF-IDF: Transforma texto em números
# LogisticRegression: Classificador (usamos class_weight='balanced' pois há muitos positivos)
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1,2))), # Unigramas e Bigramas
    ('clf', LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000))
])

print("Treinando o modelo...")
pipeline.fit(X_train, y_train)

# 4. Avaliação
print("Avaliando o modelo...")
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

# Teste manual rápido
exemplos = [
    "O produto é merda, chegou com defeito!",
    "Péssimo, veio quebrado e o atendimento não responde.",
    "É razoável, funciona mas poderia ser mais barato."
]
preds = pipeline.predict(exemplos)
probs = pipeline.predict_proba(exemplos)

print("\n--- Teste Prático ---")
for text, pred, prob in zip(exemplos, preds, probs):
    # A probabilidade retorna [prob_neg, prob_pos]. Pegamos a maior.
    confianca = max(prob)
    print(f"Texto: '{text}' -> Previsão: {pred} (Confiança: {confianca:.2f})")

# 5. Serialização (Salvar o modelo)
model_filename = 'sentiment_model.pkl'
joblib.dump(pipeline, model_filename)
print(f"\nModelo salvo como '{model_filename}'. Pronto para a API!")

Carregando dados...
Distribuição das classes:
sentiment
Positivo    16806
Negativo     1675
Name: count, dtype: int64
Treinando o modelo...
Avaliando o modelo...
              precision    recall  f1-score   support

    Negativo       0.49      0.79      0.61       335
    Positivo       0.98      0.92      0.95      3362

    accuracy                           0.91      3697
   macro avg       0.74      0.85      0.78      3697
weighted avg       0.93      0.91      0.92      3697


--- Teste Prático ---
Texto: 'O produto é merda, chegou com defeito!' -> Previsão: Negativo (Confiança: 0.72)
Texto: 'Péssimo, veio quebrado e o atendimento não responde.' -> Previsão: Negativo (Confiança: 0.96)
Texto: 'É razoável, funciona mas poderia ser mais barato.' -> Previsão: Positivo (Confiança: 0.72)

Modelo salvo como 'sentiment_model.pkl'. Pronto para a API!


In [28]:
# Teste manual rápido
exemplos = [
    "Está muito bom, chegou top++++!",
    "Péssimo, veio quebrado e o atendimento não responde 000000.",
    "Lixo, não funciona e está quebrado, ruim."
]
preds = pipeline.predict(exemplos)
probs = pipeline.predict_proba(exemplos)

print("\n--- Teste Prático ---")
for text, pred, prob in zip(exemplos, preds, probs):
    # A probabilidade retorna [prob_neg, prob_pos]. Pegamos a maior.
    confianca = max(prob)
    print(f"Texto: '{text}' -> Previsão: {pred} (Confiança: {confianca:.2f})")


--- Teste Prático ---
Texto: 'Está muito bom, chegou top++++!' -> Previsão: Positivo (Confiança: 0.92)
Texto: 'Péssimo, veio quebrado e o atendimento não responde 000000.' -> Previsão: Negativo (Confiança: 0.96)
Texto: 'Lixo, não funciona e está quebrado, ruim.' -> Previsão: Negativo (Confiança: 0.98)
